In [1]:
%matplotlib inline

In [2]:
cd ..

C:\Projects\python\recommender


In [3]:
from functools import partial
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [4]:
from datasets import SeqKaggle
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss, simple_weight_loss, trans_weight_loss

In [5]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
NEG_SAMPLE = 5
item_path = Path("./inputs/kaggle/item.csv")

In [6]:
db = SeqKaggle(data_path=item_path, user_min=4)
db

Raw dataframe shape (476244, 8)
After drop nan shape: (429988, 8)
Original comptition size: 292
Original competitor size: 140065
Filtered competiter size: 27449
Filtered dataframe shape: (284806, 8)


In [7]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE,
             neg_sample=NEG_SAMPLE)

In [8]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

## Create Criterion

In [9]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [10]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x000001DBC091FC80>, 1, 1)

In [11]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x000001DBC097BEA0>, 1, 1, 1)

In [12]:
simple_weight_loss_callback = partial(simple_weight_loss, LINEAR_REG, EMB_REG)
simple_weight_loss_callback

functools.partial(<function simple_weight_loss at 0x000001DBC097BF28>, 1, 1)

In [13]:
trans_weight_loss_callback = partial(trans_weight_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_weight_loss_callback

functools.partial(<function trans_weight_loss at 0x000001DBC0982048>, 1, 1, 1)

## Train Model

### Hyper-parameter

In [14]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

### Train FM Model

In [15]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [22]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [23]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt, step_size=DECAY_FREQ, gamma=DECAY_GAMMA)

In [24]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [17]:
fm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [19]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_weight_loss_callback)

In [25]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [27]:
fm_learner.fit(epoch=2,
               log_dir=get_log_dir(ds_type='simple_kaggle', model_type='fm'))


  0%|                                                                                                                                                                                     | 0/2 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 11795.005449829247
Epoch 0 step 1: training accuarcy: 0.9019
Epoch 0 step 1: training loss: 11262.761859629521
Epoch 0 step 2: training accuarcy: 0.9268000000000001
Epoch 0 step 2: training loss: 10840.89296971423
Epoch 0 step 3: training accuarcy: 0.9382
Epoch 0 step 3: training loss: 10407.067260047588
Epoch 0 step 4: training accuarcy: 0.9475
Epoch 0 step 4: training loss: 10139.016290808704
Epoch 0 step 5: training accuarcy: 0.9457000000000001
Epoch 0 step 5: training loss: 9820.742045991497
Epoch 0 step 6: training accuarcy: 0.9499000000000001
Epoch 0 step 6: training loss: 9497.25482930984
Epoch 0 step 7: training accuarcy: 0.9580000000000001
Epoch 0 step 7: training loss: 9192.469390586777
Epoch 0 step 8: training accuarcy: 0.9617
Epoch 0 step 8: training loss: 8897.902274778286
Epoch 0 step 9: training accuarcy: 0.9686
Epoch 0 step 9: training loss: 8644.504184018338
Epoch 0 step 10: training accuarcy: 0.9700000000000001
Epoch 0 step 10: 


 50%|██████████████████████████████████████████████████████████████████████████████████████                                                                                      | 1/2 [11:10<11:10, 670.10s/it]

Epoch: 1
Epoch 1 step 115: training loss: 393.6084775316136
Epoch 1 step 116: training accuarcy: 0.9997
Epoch 1 step 116: training loss: 383.87116867256964
Epoch 1 step 117: training accuarcy: 0.9998
Epoch 1 step 117: training loss: 370.6599519735693
Epoch 1 step 118: training accuarcy: 0.9998
Epoch 1 step 118: training loss: 369.7776940718562
Epoch 1 step 119: training accuarcy: 0.9996
Epoch 1 step 119: training loss: 355.76265512271584
Epoch 1 step 120: training accuarcy: 1.0
Epoch 1 step 120: training loss: 349.84685843749446
Epoch 1 step 121: training accuarcy: 0.9996
Epoch 1 step 121: training loss: 341.7259244850197
Epoch 1 step 122: training accuarcy: 0.9997
Epoch 1 step 122: training loss: 342.2537523285955
Epoch 1 step 123: training accuarcy: 0.9995
Epoch 1 step 123: training loss: 324.5306323480828
Epoch 1 step 124: training accuarcy: 0.9997
Epoch 1 step 124: training loss: 322.34660816559045
Epoch 1 step 125: training accuarcy: 0.9996
Epoch 1 step 125: training loss: 312.632


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [22:14<00:00, 668.42s/it]

In [21]:
del fm_model
T.cuda.empty_cache()

### Train HRM FM Model

In [22]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [23]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [24]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [28]:
hrm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [19]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_loss_callback)

In [25]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_weight_loss_callback)

In [26]:
hrm_learner.fit(epoch=2,
                log_dir=get_log_dir('simple_kaggle', 'hrm'))

  0%|                                                                                                                                                                                     | 0/2 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 491836.69427906047
Epoch 0 step 1: training accuarcy: 0.1431
Epoch 0 step 1: training loss: 492699.535647072
Epoch 0 step 2: training accuarcy: 0.1641
Epoch 0 step 2: training loss: 489907.8437827124
Epoch 0 step 3: training accuarcy: 0.1915
Epoch 0 step 3: training loss: 489669.58815967327
Epoch 0 step 4: training accuarcy: 0.19940000000000002
Epoch 0 step 4: training loss: 488313.8969329584
Epoch 0 step 5: training accuarcy: 0.21230000000000002
Epoch 0 step 5: training loss: 484669.38869526685
Epoch 0 step 6: training accuarcy: 0.22310000000000002
Epoch 0 step 6: training loss: 488643.7154459951
Epoch 0 step 7: training accuarcy: 0.226
Epoch 0 step 7: training loss: 488228.2421975789
Epoch 0 step 8: training accuarcy: 0.2253
Epoch 0 step 8: training loss: 487120.4144884759
Epoch 0 step 9: training accuarcy: 0.22060000000000002
Epoch 0 step 9: training loss: 486551.17407477234
Epoch 0 step 10: training accuarcy: 0.2286
Epoch 0 step 10: training 

KeyboardInterrupt: 

In [27]:
hrm_learner.fit(epoch=2,
                log_dir=get_log_dir('weight_kaggle', 'hrm'))


  0%|                                                                                                                                                                                     | 0/2 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 476156.54998684255
Epoch 0 step 1: training accuarcy: 0.24070000000000003
Epoch 0 step 1: training loss: 475096.2517683112
Epoch 0 step 2: training accuarcy: 0.2451
Epoch 0 step 2: training loss: 476683.3893662889
Epoch 0 step 3: training accuarcy: 0.2388
Epoch 0 step 3: training loss: 476621.68926722434
Epoch 0 step 4: training accuarcy: 0.2424
Epoch 0 step 4: training loss: 474116.48489528266
Epoch 0 step 5: training accuarcy: 0.24250000000000002
Epoch 0 step 5: training loss: 472665.09536336595
Epoch 0 step 6: training accuarcy: 0.2479
Epoch 0 step 6: training loss: 478215.3165373088
Epoch 0 step 7: training accuarcy: 0.23240000000000002
Epoch 0 step 7: training loss: 476077.3696475439
Epoch 0 step 8: training accuarcy: 0.2441
Epoch 0 step 8: training loss: 473822.3694914146
Epoch 0 step 9: training accuarcy: 0.2459
Epoch 0 step 9: training loss: 476906.9368236094
Epoch 0 step 10: training accuarcy: 0.2399
Epoch 0 step 10: training loss: 47809

Epoch 0 step 84: training loss: 456295.89030583657
Epoch 0 step 85: training accuarcy: 0.28550000000000003
Epoch 0 step 85: training loss: 462609.0593516595
Epoch 0 step 86: training accuarcy: 0.27640000000000003
Epoch 0 step 86: training loss: 457376.88746239943
Epoch 0 step 87: training accuarcy: 0.2858
Epoch 0 step 87: training loss: 458630.3304057371
Epoch 0 step 88: training accuarcy: 0.28600000000000003
Epoch 0 step 88: training loss: 457845.20144336956
Epoch 0 step 89: training accuarcy: 0.28400000000000003
Epoch 0 step 89: training loss: 463652.0738980753
Epoch 0 step 90: training accuarcy: 0.27440000000000003
Epoch 0 step 90: training loss: 462923.12193969113
Epoch 0 step 91: training accuarcy: 0.2742
Epoch 0 step 91: training loss: 462083.0978537885
Epoch 0 step 92: training accuarcy: 0.27540000000000003
Epoch 0 step 92: training loss: 453528.07437427534
Epoch 0 step 93: training accuarcy: 0.2914
Epoch 0 step 93: training loss: 457608.1776239768
Epoch 0 step 94: training accu


 50%|██████████████████████████████████████████████████████████████████████████████████████                                                                                      | 1/2 [11:03<11:03, 663.57s/it]

Epoch: 1
Epoch 1 step 115: training loss: 450378.8645166672
Epoch 1 step 116: training accuarcy: 0.29700000000000004
Epoch 1 step 116: training loss: 458080.3530401238
Epoch 1 step 117: training accuarcy: 0.2836
Epoch 1 step 117: training loss: 456139.74620575464
Epoch 1 step 118: training accuarcy: 0.2854
Epoch 1 step 118: training loss: 459225.96323987877
Epoch 1 step 119: training accuarcy: 0.2836
Epoch 1 step 119: training loss: 463845.515827573
Epoch 1 step 120: training accuarcy: 0.2772
Epoch 1 step 120: training loss: 461358.83543013706
Epoch 1 step 121: training accuarcy: 0.2771
Epoch 1 step 121: training loss: 458276.1446818058
Epoch 1 step 122: training accuarcy: 0.28040000000000004
Epoch 1 step 122: training loss: 462817.93154526915
Epoch 1 step 123: training accuarcy: 0.27940000000000004
Epoch 1 step 123: training loss: 455134.1138683906
Epoch 1 step 124: training accuarcy: 0.2898
Epoch 1 step 124: training loss: 466948.87889836164
Epoch 1 step 125: training accuarcy: 0.272

Epoch 1 step 198: training loss: 453849.4865175936
Epoch 1 step 199: training accuarcy: 0.29100000000000004
Epoch 1 step 199: training loss: 454847.26430594985
Epoch 1 step 200: training accuarcy: 0.2914
Epoch 1 step 200: training loss: 452340.755908188
Epoch 1 step 201: training accuarcy: 0.2909
Epoch 1 step 201: training loss: 449426.9151146434
Epoch 1 step 202: training accuarcy: 0.2987
Epoch 1 step 202: training loss: 454528.04437924037
Epoch 1 step 203: training accuarcy: 0.2896
Epoch 1 step 203: training loss: 455821.61266889266
Epoch 1 step 204: training accuarcy: 0.28900000000000003
Epoch 1 step 204: training loss: 451911.4399040594
Epoch 1 step 205: training accuarcy: 0.29300000000000004
Epoch 1 step 205: training loss: 448254.6464454324
Epoch 1 step 206: training accuarcy: 0.29910000000000003
Epoch 1 step 206: training loss: 450619.50617974723
Epoch 1 step 207: training accuarcy: 0.2969
Epoch 1 step 207: training loss: 455622.871717773
Epoch 1 step 208: training accuarcy: 0.2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [22:03<00:00, 662.51s/it]

In [28]:
del hrm_model
T.cuda.empty_cache()

### Train PRME FM Model

In [35]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [36]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [37]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [34]:
prme_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [32]:
prme_learner.compile(train_col='seq',
                     valid_col='seq',
                     test_col='seq',
                     loss_callback=simple_loss_callback)

In [38]:
prme_learner.compile(train_col='seq',
                     valid_col='seq',
                     test_col='seq',
                     loss_callback=simple_weight_loss_callback)

In [33]:
prme_learner.fit(epoch=2, log_dir=get_log_dir('simple_kaggle', 'prme'))


  0%|                                                                                                                                                                                     | 0/2 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 185907.28668004088
Epoch 0 step 1: training accuarcy: 0.12150000000000001
Epoch 0 step 1: training loss: 185854.96380865783
Epoch 0 step 2: training accuarcy: 0.1522
Epoch 0 step 2: training loss: 184933.9662284925
Epoch 0 step 3: training accuarcy: 0.1831
Epoch 0 step 3: training loss: 183437.2582335082
Epoch 0 step 4: training accuarcy: 0.2101
Epoch 0 step 4: training loss: 183257.959741619
Epoch 0 step 5: training accuarcy: 0.2184
Epoch 0 step 5: training loss: 182150.9192518685
Epoch 0 step 6: training accuarcy: 0.23020000000000002
Epoch 0 step 6: training loss: 184614.11063343473
Epoch 0 step 7: training accuarcy: 0.2215
Epoch 0 step 7: training loss: 183668.9743565071
Epoch 0 step 8: training accuarcy: 0.22410000000000002
Epoch 0 step 8: training loss: 184832.76475309592
Epoch 0 step 9: training accuarcy: 0.22010000000000002
Epoch 0 step 9: training loss: 181807.46099861656
Epoch 0 step 10: training accuarcy: 0.23290000000000002
Epoch 0 ste

Epoch 0 step 83: training accuarcy: 0.24200000000000002
Epoch 0 step 83: training loss: 174757.0904206139
Epoch 0 step 84: training accuarcy: 0.2341
Epoch 0 step 84: training loss: 173827.0898328011
Epoch 0 step 85: training accuarcy: 0.2389
Epoch 0 step 85: training loss: 171825.06579376027
Epoch 0 step 86: training accuarcy: 0.2494
Epoch 0 step 86: training loss: 173857.12502389672
Epoch 0 step 87: training accuarcy: 0.24020000000000002
Epoch 0 step 87: training loss: 173202.98805840008
Epoch 0 step 88: training accuarcy: 0.2432
Epoch 0 step 88: training loss: 172927.33162710498
Epoch 0 step 89: training accuarcy: 0.2442
Epoch 0 step 89: training loss: 171977.56767326145
Epoch 0 step 90: training accuarcy: 0.2482
Epoch 0 step 90: training loss: 173432.23474081184
Epoch 0 step 91: training accuarcy: 0.24150000000000002
Epoch 0 step 91: training loss: 172398.09092915198
Epoch 0 step 92: training accuarcy: 0.2461
Epoch 0 step 92: training loss: 172844.4220802758
Epoch 0 step 93: trainin


 50%|██████████████████████████████████████████████████████████████████████████████████████                                                                                      | 1/2 [11:27<11:27, 687.96s/it]

Epoch: 1
Epoch 1 step 115: training loss: 173367.75662909678
Epoch 1 step 116: training accuarcy: 0.2401
Epoch 1 step 116: training loss: 172121.59036878622
Epoch 1 step 117: training accuarcy: 0.2482
Epoch 1 step 117: training loss: 173394.39221695228
Epoch 1 step 118: training accuarcy: 0.2393
Epoch 1 step 118: training loss: 172953.98294053925
Epoch 1 step 119: training accuarcy: 0.2427
Epoch 1 step 119: training loss: 174123.6051510823
Epoch 1 step 120: training accuarcy: 0.2374
Epoch 1 step 120: training loss: 172814.79981924684
Epoch 1 step 121: training accuarcy: 0.24350000000000002
Epoch 1 step 121: training loss: 172134.87371858204
Epoch 1 step 122: training accuarcy: 0.24480000000000002
Epoch 1 step 122: training loss: 172971.76418508976
Epoch 1 step 123: training accuarcy: 0.24080000000000001
Epoch 1 step 123: training loss: 175290.1779008029
Epoch 1 step 124: training accuarcy: 0.2305
Epoch 1 step 124: training loss: 172473.34178168638
Epoch 1 step 125: training accuarcy: 0

Epoch 1 step 198: training loss: 162610.53283019626
Epoch 1 step 199: training accuarcy: 0.2891
Epoch 1 step 199: training loss: 162199.46468374698
Epoch 1 step 200: training accuarcy: 0.2898
Epoch 1 step 200: training loss: 162427.2045080176
Epoch 1 step 201: training accuarcy: 0.2886
Epoch 1 step 201: training loss: 164564.78317812993
Epoch 1 step 202: training accuarcy: 0.28140000000000004
Epoch 1 step 202: training loss: 161991.48053854122
Epoch 1 step 203: training accuarcy: 0.29250000000000004
Epoch 1 step 203: training loss: 164103.1041296549
Epoch 1 step 204: training accuarcy: 0.28150000000000003
Epoch 1 step 204: training loss: 163178.19183562676
Epoch 1 step 205: training accuarcy: 0.2874
Epoch 1 step 205: training loss: 163649.64187065902
Epoch 1 step 206: training accuarcy: 0.28450000000000003
Epoch 1 step 206: training loss: 159755.57006093411
Epoch 1 step 207: training accuarcy: 0.30110000000000003
Epoch 1 step 207: training loss: 161762.42751478814
Epoch 1 step 208: tra


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [22:57<00:00, 688.37s/it]

In [39]:
prme_learner.fit(epoch=2, log_dir=get_log_dir('weight_kaggle', 'prme'))


  0%|                                                                                                                                                                                     | 0/2 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 487422.82394555013
Epoch 0 step 1: training accuarcy: 0.12040000000000001
Epoch 0 step 1: training loss: 489450.34019803803
Epoch 0 step 2: training accuarcy: 0.1499
Epoch 0 step 2: training loss: 488016.4392187835
Epoch 0 step 3: training accuarcy: 0.1763
Epoch 0 step 3: training loss: 491882.392495647
Epoch 0 step 4: training accuarcy: 0.1903
Epoch 0 step 4: training loss: 490320.818300761
Epoch 0 step 5: training accuarcy: 0.2059
Epoch 0 step 5: training loss: 490525.54563383816
Epoch 0 step 6: training accuarcy: 0.215
Epoch 0 step 6: training loss: 485183.02029845637
Epoch 0 step 7: training accuarcy: 0.22240000000000001
Epoch 0 step 7: training loss: 487375.3059561003
Epoch 0 step 8: training accuarcy: 0.2267
Epoch 0 step 8: training loss: 489671.4944303962
Epoch 0 step 9: training accuarcy: 0.22710000000000002
Epoch 0 step 9: training loss: 492945.4432845665
Epoch 0 step 10: training accuarcy: 0.21630000000000002
Epoch 0 step 10: training l

Epoch 0 step 83: training loss: 473360.3601560681
Epoch 0 step 84: training accuarcy: 0.2485
Epoch 0 step 84: training loss: 473256.6977034972
Epoch 0 step 85: training accuarcy: 0.24710000000000001
Epoch 0 step 85: training loss: 475983.0429531854
Epoch 0 step 86: training accuarcy: 0.24180000000000001
Epoch 0 step 86: training loss: 474157.9603834493
Epoch 0 step 87: training accuarcy: 0.2464
Epoch 0 step 87: training loss: 482785.3637185157
Epoch 0 step 88: training accuarcy: 0.234
Epoch 0 step 88: training loss: 470584.06494812225
Epoch 0 step 89: training accuarcy: 0.2482
Epoch 0 step 89: training loss: 474801.79766546725
Epoch 0 step 90: training accuarcy: 0.2479
Epoch 0 step 90: training loss: 476233.41311513004
Epoch 0 step 91: training accuarcy: 0.2422
Epoch 0 step 91: training loss: 471155.5216672345
Epoch 0 step 92: training accuarcy: 0.2492
Epoch 0 step 92: training loss: 473965.4642374708
Epoch 0 step 93: training accuarcy: 0.2432
Epoch 0 step 93: training loss: 475442.202


 50%|██████████████████████████████████████████████████████████████████████████████████████                                                                                      | 1/2 [11:09<11:09, 669.07s/it]

Epoch: 1
Epoch 1 step 115: training loss: 474542.40284475335
Epoch 1 step 116: training accuarcy: 0.24000000000000002
Epoch 1 step 116: training loss: 473252.3797201521
Epoch 1 step 117: training accuarcy: 0.24650000000000002
Epoch 1 step 117: training loss: 477294.37638240063
Epoch 1 step 118: training accuarcy: 0.2396
Epoch 1 step 118: training loss: 476808.94339811546
Epoch 1 step 119: training accuarcy: 0.2421
Epoch 1 step 119: training loss: 471741.0805636938
Epoch 1 step 120: training accuarcy: 0.2466
Epoch 1 step 120: training loss: 473419.45447076997
Epoch 1 step 121: training accuarcy: 0.2457
Epoch 1 step 121: training loss: 476231.16271982976
Epoch 1 step 122: training accuarcy: 0.2431
Epoch 1 step 122: training loss: 478678.3387199232
Epoch 1 step 123: training accuarcy: 0.24000000000000002
Epoch 1 step 123: training loss: 476005.4940440219
Epoch 1 step 124: training accuarcy: 0.2411
Epoch 1 step 124: training loss: 475530.66795238934
Epoch 1 step 125: training accuarcy: 0.2

Epoch 1 step 196: training loss: 474832.1428100941
Epoch 1 step 197: training accuarcy: 0.2441
Epoch 1 step 197: training loss: 470904.4547174661
Epoch 1 step 198: training accuarcy: 0.248
Epoch 1 step 198: training loss: 476812.9139500183
Epoch 1 step 199: training accuarcy: 0.2411
Epoch 1 step 199: training loss: 476426.25979537575
Epoch 1 step 200: training accuarcy: 0.23970000000000002
Epoch 1 step 200: training loss: 473637.82748101675
Epoch 1 step 201: training accuarcy: 0.24480000000000002
Epoch 1 step 201: training loss: 471086.9820538843
Epoch 1 step 202: training accuarcy: 0.24760000000000001
Epoch 1 step 202: training loss: 480343.3390708109
Epoch 1 step 203: training accuarcy: 0.23750000000000002
Epoch 1 step 203: training loss: 475530.0101194482
Epoch 1 step 204: training accuarcy: 0.2487
Epoch 1 step 204: training loss: 471033.1300218787
Epoch 1 step 205: training accuarcy: 0.2482
Epoch 1 step 205: training loss: 471966.6060932759
Epoch 1 step 206: training accuarcy: 0.25


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [22:23<00:00, 670.79s/it]

In [40]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [48]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [49]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [50]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [40]:
trans_learner.compile(train_col='base',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=simple_loss_callback)

In [45]:
trans_learner.compile(train_col='seq',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=simple_loss_callback)

In [51]:
trans_learner.compile(train_col='seq',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=simple_weight_loss_callback)

In [46]:
trans_learner.fit(epoch=2,
                  log_dir=get_log_dir('simple_kaggle', 'trans'))


  0%|                                                                                                                                                                                     | 0/2 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 13806.398639158986
Epoch 0 step 1: training accuarcy: 0.8815000000000001
Epoch 0 step 1: training loss: 13303.707887064213
Epoch 0 step 2: training accuarcy: 0.9153
Epoch 0 step 2: training loss: 12815.493552099597
Epoch 0 step 3: training accuarcy: 0.9420000000000001
Epoch 0 step 3: training loss: 12345.804191474945
Epoch 0 step 4: training accuarcy: 0.9599000000000001
Epoch 0 step 4: training loss: 11859.877499910504
Epoch 0 step 5: training accuarcy: 0.9687
Epoch 0 step 5: training loss: 11405.112049870202
Epoch 0 step 6: training accuarcy: 0.9733
Epoch 0 step 6: training loss: 10962.206704302316
Epoch 0 step 7: training accuarcy: 0.9766
Epoch 0 step 7: training loss: 10561.753157123152
Epoch 0 step 8: training accuarcy: 0.9771000000000001
Epoch 0 step 8: training loss: 10169.054863215862
Epoch 0 step 9: training accuarcy: 0.9764
Epoch 0 step 9: training loss: 9814.151412577852
Epoch 0 step 10: training accuarcy: 0.9774
Epoch 0 step 10: traini

Epoch 0 step 83: training accuarcy: 0.9921000000000001
Epoch 0 step 83: training loss: 755.7244235455796
Epoch 0 step 84: training accuarcy: 0.9916
Epoch 0 step 84: training loss: 714.1708805339747
Epoch 0 step 85: training accuarcy: 0.9923000000000001
Epoch 0 step 85: training loss: 684.6375754587165
Epoch 0 step 86: training accuarcy: 0.9933000000000001
Epoch 0 step 86: training loss: 698.6962445972293
Epoch 0 step 87: training accuarcy: 0.9907
Epoch 0 step 87: training loss: 674.8777554267851
Epoch 0 step 88: training accuarcy: 0.9906
Epoch 0 step 88: training loss: 645.7831004724978
Epoch 0 step 89: training accuarcy: 0.9925
Epoch 0 step 89: training loss: 624.1942273867717
Epoch 0 step 90: training accuarcy: 0.9926
Epoch 0 step 90: training loss: 614.444426267431
Epoch 0 step 91: training accuarcy: 0.9903000000000001
Epoch 0 step 91: training loss: 587.1531958147577
Epoch 0 step 92: training accuarcy: 0.993
Epoch 0 step 92: training loss: 561.3538307026868
Epoch 0 step 93: trainin


 50%|██████████████████████████████████████████████████████████████████████████████████████                                                                                      | 1/2 [11:09<11:09, 669.73s/it]

Epoch: 1
Epoch 1 step 115: training loss: 318.6883368290194
Epoch 1 step 116: training accuarcy: 0.9943000000000001
Epoch 1 step 116: training loss: 330.6143448810297
Epoch 1 step 117: training accuarcy: 0.9927
Epoch 1 step 117: training loss: 328.35024013344776
Epoch 1 step 118: training accuarcy: 0.9918
Epoch 1 step 118: training loss: 312.9796494906412
Epoch 1 step 119: training accuarcy: 0.9925
Epoch 1 step 119: training loss: 299.61080719406596
Epoch 1 step 120: training accuarcy: 0.9936
Epoch 1 step 120: training loss: 275.24004544813397
Epoch 1 step 121: training accuarcy: 0.9954000000000001
Epoch 1 step 121: training loss: 296.7289028365274
Epoch 1 step 122: training accuarcy: 0.9935
Epoch 1 step 122: training loss: 292.669406522361
Epoch 1 step 123: training accuarcy: 0.9926
Epoch 1 step 123: training loss: 273.0930750524542
Epoch 1 step 124: training accuarcy: 0.9937
Epoch 1 step 124: training loss: 280.5020568114061
Epoch 1 step 125: training accuarcy: 0.9928
Epoch 1 step 12

Epoch 1 step 197: training loss: 178.9023833051987
Epoch 1 step 198: training accuarcy: 0.9933000000000001
Epoch 1 step 198: training loss: 168.30235408147863
Epoch 1 step 199: training accuarcy: 0.9948
Epoch 1 step 199: training loss: 162.7889748578452
Epoch 1 step 200: training accuarcy: 0.9946
Epoch 1 step 200: training loss: 164.77832768623597
Epoch 1 step 201: training accuarcy: 0.9947
Epoch 1 step 201: training loss: 182.24448627529296
Epoch 1 step 202: training accuarcy: 0.9932000000000001
Epoch 1 step 202: training loss: 168.18048031785122
Epoch 1 step 203: training accuarcy: 0.9947
Epoch 1 step 203: training loss: 176.313799077576
Epoch 1 step 204: training accuarcy: 0.9935
Epoch 1 step 204: training loss: 160.91246318041846
Epoch 1 step 205: training accuarcy: 0.9951000000000001
Epoch 1 step 205: training loss: 168.94934256112273
Epoch 1 step 206: training accuarcy: 0.9941000000000001
Epoch 1 step 206: training loss: 165.9653310506707
Epoch 1 step 207: training accuarcy: 0.99


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [22:18<00:00, 669.58s/it]

In [52]:
trans_learner.fit(epoch=2, log_dir=get_log_dir('weight_kaggle', 'trans'))


  0%|                                                                                                                                                                                     | 0/2 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 17381.002641464496
Epoch 0 step 1: training accuarcy: 0.8965000000000001
Epoch 0 step 1: training loss: 16496.53774284842
Epoch 0 step 2: training accuarcy: 0.92
Epoch 0 step 2: training loss: 15744.861349360843
Epoch 0 step 3: training accuarcy: 0.9458000000000001
Epoch 0 step 3: training loss: 14956.835339405248
Epoch 0 step 4: training accuarcy: 0.9612
Epoch 0 step 4: training loss: 14224.434288367096
Epoch 0 step 5: training accuarcy: 0.9690000000000001
Epoch 0 step 5: training loss: 13756.064527740442
Epoch 0 step 6: training accuarcy: 0.9709000000000001
Epoch 0 step 6: training loss: 12986.189324055866
Epoch 0 step 7: training accuarcy: 0.9756
Epoch 0 step 7: training loss: 12541.985667730762
Epoch 0 step 8: training accuarcy: 0.9728
Epoch 0 step 8: training loss: 12022.593255255164
Epoch 0 step 9: training accuarcy: 0.9726
Epoch 0 step 9: training loss: 11626.595902241628
Epoch 0 step 10: training accuarcy: 0.9755
Epoch 0 step 10: training

Epoch 0 step 82: training loss: 1189.7284796752924
Epoch 0 step 83: training accuarcy: 0.9908
Epoch 0 step 83: training loss: 1086.96257575096
Epoch 0 step 84: training accuarcy: 0.9937
Epoch 0 step 84: training loss: 1116.573634323385
Epoch 0 step 85: training accuarcy: 0.9922000000000001
Epoch 0 step 85: training loss: 1114.9663954367995
Epoch 0 step 86: training accuarcy: 0.9905
Epoch 0 step 86: training loss: 1067.1138154366363
Epoch 0 step 87: training accuarcy: 0.992
Epoch 0 step 87: training loss: 1005.6448374576487
Epoch 0 step 88: training accuarcy: 0.9931000000000001
Epoch 0 step 88: training loss: 988.8289004242874
Epoch 0 step 89: training accuarcy: 0.9926
Epoch 0 step 89: training loss: 1045.8426020888674
Epoch 0 step 90: training accuarcy: 0.9913000000000001
Epoch 0 step 90: training loss: 1001.5470468685141
Epoch 0 step 91: training accuarcy: 0.9915
Epoch 0 step 91: training loss: 983.6988965070402
Epoch 0 step 92: training accuarcy: 0.9919
Epoch 0 step 92: training loss


 50%|██████████████████████████████████████████████████████████████████████████████████████                                                                                      | 1/2 [11:15<11:15, 675.75s/it]

Epoch: 1
Epoch 1 step 115: training loss: 664.9170099245275
Epoch 1 step 116: training accuarcy: 0.9927
Epoch 1 step 116: training loss: 599.046155383268
Epoch 1 step 117: training accuarcy: 0.9926
Epoch 1 step 117: training loss: 615.8627759908185
Epoch 1 step 118: training accuarcy: 0.9916
Epoch 1 step 118: training loss: 631.6514158098196
Epoch 1 step 119: training accuarcy: 0.9922000000000001
Epoch 1 step 119: training loss: 657.8047752044704
Epoch 1 step 120: training accuarcy: 0.9911000000000001
Epoch 1 step 120: training loss: 606.0884341488816
Epoch 1 step 121: training accuarcy: 0.9927
Epoch 1 step 121: training loss: 530.9986929363481
Epoch 1 step 122: training accuarcy: 0.9939
Epoch 1 step 122: training loss: 563.6020754520605
Epoch 1 step 123: training accuarcy: 0.9926
Epoch 1 step 123: training loss: 556.0102186648044
Epoch 1 step 124: training accuarcy: 0.9933000000000001
Epoch 1 step 124: training loss: 534.6703722445317
Epoch 1 step 125: training accuarcy: 0.9935
Epoch 

Epoch 1 step 197: training accuarcy: 0.9935
Epoch 1 step 197: training loss: 345.6891099117653
Epoch 1 step 198: training accuarcy: 0.9964000000000001
Epoch 1 step 198: training loss: 378.44271589610366
Epoch 1 step 199: training accuarcy: 0.9944000000000001
Epoch 1 step 199: training loss: 378.5350091680062
Epoch 1 step 200: training accuarcy: 0.9957
Epoch 1 step 200: training loss: 389.882784343892
Epoch 1 step 201: training accuarcy: 0.9941000000000001
Epoch 1 step 201: training loss: 337.50168711458946
Epoch 1 step 202: training accuarcy: 0.9963000000000001
Epoch 1 step 202: training loss: 389.4433368269072
Epoch 1 step 203: training accuarcy: 0.994
Epoch 1 step 203: training loss: 356.5700536028768
Epoch 1 step 204: training accuarcy: 0.9952000000000001
Epoch 1 step 204: training loss: 417.68784188598306
Epoch 1 step 205: training accuarcy: 0.9935
Epoch 1 step 205: training loss: 371.4368643084421
Epoch 1 step 206: training accuarcy: 0.9941000000000001
Epoch 1 step 206: training l


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [22:26<00:00, 674.30s/it]

In [53]:
del trans_model
T.cuda.empty_cache()